In [20]:
#construct all csfs on site 1 and site 2 (extra electron on site 2)
#site 1
sextet=[0.5,1,1.5,2,2.5]
quartet1=[0.5,1,1.5,2,1.5]
quartet2=[0.5,1,1.5,1,1.5]
quartet3=[0.5,1,0.5,1,1.5]
quartet4=[0.5,0,0.5,1,1.5]
doublet1=[0.5,1,0.5,1,0.5]
doublet2=[0.5,1,0.5,0,0.5]
doublet3=[0.5,0,0.5,1,0.5]
doublet4=[0.5,0,0.5,0,0.5]
doublet5=[0.5,1,1.5,1,0.5]

#site2
quintet=[0.5,1,1.5,2]
triplet1=[0.5,1,1.5,1]
triplet2=[0.5,1,0.5,1]
triplet3=[0.5,0,0.5,1]
singlet1=[0.5,1,0.5,0]
singlet2=[0.5,0,0.5,0]

#genealog to j label (list)
#print(quintet[-1])


In [21]:
#generates all permutations of slater determinants with a given string of a,b and c orbitals. 
#a=alpha b=beta and c=closed shell, doubly occupied
from itertools import permutations
import math

def unique_perms(string):
    return {"".join(p) for p in permutations(string)}
    
def state(string):
    vac=[]
    for i in range(len(unique_perms(string))):
        vac.append(list(sorted(unique_perms(string))[i]))
    return vac
print(state('aaaab'))

#define a dictionary that takes m values and gives a string of a,b,c with the same m
spin_projection={2.5:'aaaaa',2:'aaaac',1.5:'aaaab',1:'aaabc',0.5:'aaabb',0:'aabbc',-0.5:'aabbb',-1:'abbbc',-1.5:'abbbb',-2:'bbbbc',-2.5:'bbbbb'}

[['a', 'a', 'a', 'a', 'b'], ['a', 'a', 'a', 'b', 'a'], ['a', 'a', 'b', 'a', 'a'], ['a', 'b', 'a', 'a', 'a'], ['b', 'a', 'a', 'a', 'a']]


In [22]:
#converts a SD with a,b,c labelled orbitals into the genealogical coupled Mz states
def genealog(lst):
    dictionary={'a':0.5,'b':-0.5}
    arr=[]
    if 'c' in lst:
        lst.remove('c')
        for i in range(len(lst)):
            if i==0:
                arr.append(dictionary.get(lst[i]))
            else:
                arr.append(dictionary.get(lst[i])+arr[i-1])
        return arr
    else:
        for i in range(len(lst)):
            if i==0:
                arr.append(dictionary.get(lst[i]))
            else:
                arr.append(dictionary.get(lst[i])+arr[i-1])
        return arr
#print(genealog(['a', 'a', 'a', 'a', 'c']))
#print(get_total_coupling_coefficient(genealog(['a', 'a', 'a', 'a']),genealog([ 'a', 'a', 'a','a'])))

In [23]:
#Clebsch_gordan coupling of CSF sites
from sympy import S
from sympy.physics.wigner import clebsch_gordan


def get_cg(j1, j2, j, m1, m2, m, analytic=False):
    r"""
    Get Clebsch-Gordon coefficients. Calculated using Sympy.
    :param j1: Spin of state 1
    :param j2: Spin of state 2
    :param j:  Spin of coupled state
    :param m1: Spin projection of state 1
    :param m2: Spin projection of state 2
    :param m:  Spin projection of coupled state
    :param analytic: :bool: if True, return analytic expression for the Clebsch-Gordon coefficient
    :return:   :float: Clebsch-Gordon coefficient
    """
    expr = clebsch_gordan(S(int(2 * j1)) / 2, S(int(2 * j2)) / 2, S(int(2 * j)) / 2, S(int(2 * m1)) / 2,
                          S(int(2 * m2)) / 2, S(int(2 * m)) / 2)
    if analytic:
        return expr
    else:
        return expr.evalf()


def get_general_tensorprod(j1, j2, j, m):
    r"""
    For a target COUPLED spin state of spin quantum number j with spin projection m,
    compute the necessary linear combination needed from states of spins j1 and j2
    :param j1: Spin of state 1
    :param j2: Spin of state 2
    :param j:  Spin of coupled state
    :param m:  Spin projection of coupled state
    :return:   List of List[float, float, float, float] in [j1, m1, j2, m2] of the states required for coupling
    """
    # We shall work in half-integer steps
    j1 = int(2 * j1)
    j2 = int(2 * j2)
    j = int(2 * j)
    m = int(2 * m)
    assert abs(j1 - j2) <= j <= j1 + j2, "Impossible set of spin quantum numbers"
    states_required = []
    for m1 in range(-j1, j1 + 1, 2):  # m goes in integer steps
        for m2 in range(-j2, j2 + 1, 2):
            if m1 + m2 == m:
                states_required.append([j1 / 2, m1 / 2, j2 / 2, m2 / 2])
    return states_required


def take_csf_tensorprod(kets_a, coeffs_a, kets_b, coeffs_b, cg):
    r"""
    Take the tensor product of the kets and cofficients on different sites. Multiply coefficient product by the
    Clebsch-Gordon coefficient.
    :param kets_a:      List of List[int]. List[int] has format: [pf, a, a, a, ..., b, b, ...]. pf = Phase factor,
                        a and b are alpha and beta occupations, respectively (0 for empty, 1 for filled)
    :param coeffs_a:    1D :np.ndarray: Coefficient of ket_a
    :param kets_b:      As kets_a
    :param coeffs_b:    As coeffs_a
    :param cg:          :float: Clebsch-Gordon coefficient
    :return:            List of List[int] of the coupled state
    """
    new_kets = []
    new_coeffs = []
    for a, ket_a in enumerate(kets_a):
        for b, ket_b in enumerate(kets_b):
            na = (len(ket_a)-1) // 2
            nb = (len(ket_b)-1) // 2
            pf = ket_a[0] * ket_b[0]
            new_ket = [pf] + ket_a[1:na+1] + ket_b[1:nb+1] + ket_a[na+1:] + ket_b[nb+1:]
            new_coeff = float(coeffs_a[a] * coeffs_b[b] * cg)
            new_kets.append(new_ket)
            new_coeffs.append(new_coeff)
    return new_kets, new_coeffs


def get_local_g_coupling(norbs, j):
    r"""
    Construct a genealogical coupling pattern naively.
    :param norbs: :int: Number of orbitals
    :param j:     :float: (Takes on integer or half-integer values) Spin quantum number
    :return:      :str: Corresponding to a genealogical coupling branch e.g. ++-- for V CSF
    """
    j = int(2 * j)
    ps = j
    leftovers = norbs - j
    assert leftovers % 2 == 0
    ps += leftovers // 2
    ns = leftovers // 2
    return ps * "+" + ns * "-"

In [24]:
#coupling coefficients of SDs for a given CSF
import sys
import numpy as np


def get_coupling_coefficient(Tn, Pn, tn, pn):
    r"""
    Computes the coupling coefficient C_{tn, pn}^{Tn, Pn}
    :param Tn:
    :param Pn:
    :param tn:
    :param pn:
    :return:
    """
    # This is a forbidden case
    if Tn < np.abs(Pn):
        return 0
    if np.isclose(0.5, tn, rtol=0, atol=1e-10):
        return np.sqrt((Tn + 2 * pn * Pn) / (2 * Tn))
    elif np.isclose(-0.5, tn, rtol=0, atol=1e-10):
        return -2 * pn * np.sqrt((Tn + 1 - 2 * pn * Pn) / (2 * (Tn + 1)))
    else:
        print("A coefficient requested is invalid. Exiting.")
        sys.exit(1)


def get_total_coupling_coefficient(det, csf):
    r"""
    Gets the overlap between the determinant and the CSF. This is the coefficient d of the determinant in the CSF.
    :param det:
    :param csf:
    :return:
    """
    total_coeff = 1
    assert len(det) == len(csf), "Number of orbitals in determinant and CSF are not the same. Check again."
    degeneracy={5:1,4:1/np.sqrt(5)}
    for i in range(1, len(det)):
        Tn = csf[i]
        Pn = det[i]
        tn = csf[i] - csf[i - 1]
        pn = det[i] - det[i - 1]
        total_coeff = total_coeff * get_coupling_coefficient(Tn, Pn, tn, pn)
    return total_coeff*degeneracy.get(len(det))

In [25]:
#generates total wavefunction
#print(get_general_tensorprod(sextet[-1],quintet[-1],0.5, 0.5))
#print(get_cg(sextet[-1], quintet[-1],0.5, 2.5, -2, 0.5, analytic=False))
def generate_wfctn(Fe1,Fe2,j,m):
    system=[]
    for i in range(len(get_general_tensorprod(Fe1[-1], Fe2[-1],0.5, 0.5))):
        couple=[]
        couple.append(state(spin_projection.get(get_general_tensorprod(Fe1[-1], Fe2[-1],0.5, 0.5)[i][1])))
        couple.append(state(spin_projection.get(get_general_tensorprod(Fe1[-1], Fe2[-1],0.5, 0.5)[i][3])))
        system.append(couple)
    return system
#print(generate_wfctn(sextet,quintet,0.5,0.5))   

w=generate_wfctn(sextet,quintet,0.5,0.5)

In [26]:
#generates coefficent tensor from wavefunction
# #only Fe1 coefficients are multiplied by the CG coefficient so as to account for both without sqrting -ve numbers
def generate_coeff(Fe1,Fe2,j,m):
    import math
    arr=get_general_tensorprod(Fe1[-1],Fe2[-1],0.5, 0.5)
    coeff=[]
    for i in range(len(generate_wfctn(Fe1,Fe2,j,m))):
        coupal=[]
        lst1=[]
        lst2=[]
        for k in range(len(generate_wfctn(Fe1,Fe2,j,m)[i][0])):
            lst1.append([get_total_coupling_coefficient(genealog(generate_wfctn(Fe1,Fe2,j,m)[i][0][k]),Fe1)*(get_cg(Fe1[-1], Fe2[-1],j, arr[i][1], arr[i][3], 0.5, analytic=False))])
        for l in range(len(generate_wfctn(Fe1,Fe2,j,m)[i][1])):
            lst2.append([get_total_coupling_coefficient(genealog(generate_wfctn(Fe1,Fe2,j,m)[i][1][l]),Fe2)])
        coupal.append(lst1)
        coupal.append(lst2)
        coeff.append(coupal)
    return coeff
#print(generate_coeff(sextet,quintet,0.5,0.5))
c=generate_coeff(sextet,quintet,0.5,0.5)

In [35]:
# checking to see if normalized coefficients V2
a=0
for i in range(len(c)):
    for k in range(len(c[i][0])):
        for l in range(len(c[i][1])):
            a+=(c[i][0][k][0]*c[i][1][l][0])**2
#print(a) 

In [28]:
#indexing tool to find SD in total wavefunction (need for comparing SDs)
#only works for lst_search that are all the same length
def deepest_index(lst,lst_search):
    for i in range(len(lst)):
        if all(x == y for x, y in zip(lst[i],lst_search)):
            return i

#picks out the index of a SD
def deep_index(lst,lst_search):
    for k in range(len(lst)):
        for j in range(len(lst[k])):
            if deepest_index(lst[k][j],lst_search)!=None:
                return [k,j,deepest_index(lst[k][j],lst_search)]
                
#gives value of coefficient for corresponding index
def index_coefficient(lst,lst_search,coefficients):
    if deep_index(lst,lst_search)==None:
        return 0
    else:
        return coefficients[deep_index(lst,lst_search)[0]][deep_index(lst,lst_search)[1]][deep_index(lst,lst_search)[2]][0]

#print(index_coefficient(total_wavefunction,['a','a','a','a','a'],c))

#Test 1
#list3=[[[['a','a'],['b','a',]],[['a','b'],['a','a','b']]],[[['a','d'],['d','a',]],[['a','c'],['a','a','d']]]]
#print(list3[deep_index(list3,['d','a'])[0]][deep_index(list3,['d','a'])[1]][deep_index(list3,['d','a'])[2]])
#Test 2
#print(total_wavefunction[deep_index(total_wavefunction,['a','a','a','a','a'])[0]]
      #[deep_index(total_wavefunction,['a','a','a','a','a'])[1]]
      #[deep_index(total_wavefunction,['a','a','a','a','a'])[2]])

In [29]:
#spin projection operators
string='a'
m=0
for element in string:
    if element=='a':
        m += 0.5
    elif element=='b':
        m += -0.5
    elif element=='c':
        m += 0
#print(m)

#construct the s1iz and s2iz operators as a function of any list
# sniz gives the total value of m for the ith d orbital on the nth Fe atom

def siz(i,lst):
    m=0
    if lst[i-1]=='a':
        m += 0.5
    elif lst[i-1]=='b':
        m += -0.5
    elif lst[i-1]=='c':
            m += 0
    return m
i=1
def s1iz_s2iz(i,total_wavefunction,coefficients):
    Mi=0
    for h in range(len(total_wavefunction)):
        for k in range(len(total_wavefunction[h][0])):
            for l in range(len(total_wavefunction[h][1])):
                Mi+=(siz(i,total_wavefunction[h][0][k])*siz(i,total_wavefunction[h][1][l])*
                     (coefficients[h][0][k][0]*coefficients[h][1][l][0])**2)
    return Mi

In [30]:
#raising and lowering spin operators
#constructed sni+ and sni-, operators that raise the ith (1-5) element of a list
list1=['a','b','a','a','b']
def raising_i(i,list1):
    list2=list1.copy()
    g=0
    if list1[i-1]=='b':
        list2=list1[:(i-1)]+list1[i:]
        list2.insert(i-1,'a')
        g=1
    else:
        g=0
    return [list2,g]
#print(raising_i(2,list1))
#print(list1)

def lowering_i(i,list1):
    list2=list1.copy()
    h=0
    if list1[i-1]=='a':
        list2=list1[:(i-1)]+list1[i:]
        list2.insert(i-1,'b')
        h=1
    else:
        h=0
    return [list2,h]
#print(lowering_i(2,list1))

#constructed sni+xsni-
def s1i_raising_s2i_lowering(i,total_wavefunction,coefficients):
    x=0
    for h in range(len(total_wavefunction)):
        for k in range(len(total_wavefunction[h][0])):
            for l in range(len(total_wavefunction[h][1])):
                    x+=(raising_i(i,total_wavefunction[h][0][k])[1]*lowering_i(i,total_wavefunction[h][1][l])[1]*
                        index_coefficient(total_wavefunction,total_wavefunction[h][0][k],coefficients)*
                        index_coefficient(total_wavefunction,total_wavefunction[h][1][l],coefficients)*
                        index_coefficient(total_wavefunction,raising_i(i,total_wavefunction[h][0][k])[0],coefficients)*
                        index_coefficient(total_wavefunction,lowering_i(i,total_wavefunction[h][1][l])[0],coefficients))
    return x 
    
def s1i_lowering_s2i_raising(i,total_wavefunction,coefficients):
    y=0
    for h in range(len(total_wavefunction)):
        for k in range(len(total_wavefunction[h][0])):
            for l in range(len(total_wavefunction[h][1])):
                    y+=(lowering_i(i,total_wavefunction[h][0][k])[1]*raising_i(i,total_wavefunction[h][1][l])[1]*
                        index_coefficient(total_wavefunction,total_wavefunction[h][0][k],coefficients)*
                        index_coefficient(total_wavefunction,total_wavefunction[h][1][l],coefficients)*
                        index_coefficient(total_wavefunction,lowering_i(i,total_wavefunction[h][0][k])[0],coefficients)*
                        index_coefficient(total_wavefunction,raising_i(i,total_wavefunction[h][1][l])[0],coefficients))
    return y 

In [31]:
#ligand field splitting

def splitting_i(i,lst):
    d=0
    for element in lst:
        if lst[i-1]=='a':
            d += 1/5
        elif lst[i-1]=='b':
            d += 1/5
        elif lst[i-1]=='c':
            d += 2/5
    return d
def ligand_field_splitting_i(i,total_wavefunction,coefficients):
    z=0
    for h in range(len(coefficients)):
        for k in range(len(coefficients[h][0])):
            for l in range(len(coefficients[h][1])):
                z+=((splitting_i(i,total_wavefunction[h][0][k])+splitting_i(i,total_wavefunction[h][1][l]))*
                     (coefficients[h][0][k][0]*coefficients[h][1][l][0])**2)
    return z

In [32]:
#hopping term
#annihilation and creation operators for the ith d orbital and spin1 (e.g. annihilates spin1 and creates 
#the opposite spin to that present as a variable!
def annih_i_ospin(i,list1,spin):
    list2=list1.copy()
    g=0
    if list1[i-1]=='c':
        list2=list1[:(i-1)]+list1[i:]
        list2.insert(i-1,spin)
        g=1
    else:
        g=0
    return [list2,g]
#print(annih_i_ospin(1,['c','a','b'],'b'))

def creat_i_ospin(i,list1,spin):
    list2=list1.copy()
    h=0
    if list1[i-1]==spin:
        list2=list1[:(i-1)]+list1[i:]
        list2.insert(i-1,'c')
        h=1
    else:
        h=0
    return [list2,h]
#print(creat_i_ospin(3,['c','a','b'],'b'))

#defining the combined creation and annihilation operators
#hopping term requires the wavefunction to be a linear combination of wavefunctions localised on both Fe sites
def s1i_creation_s2i_annihilation(i,total_wavefunction,coefficients,spin):
    x=0
    for h in range(len(total_wavefunction)):
        for k in range(len(total_wavefunction[h][0])):
            for l in range(len(total_wavefunction[h][1])):
                    x+=(creat_i_ospin(i,total_wavefunction[h][0][k],spin)[1]
                        *annih_i_ospin(i,total_wavefunction[h][1][l],spin)[1]*
                        index_coefficient(total_wavefunction,total_wavefunction[h][0][k],coefficients)*
                        index_coefficient(total_wavefunction,total_wavefunction[h][1][l],coefficients)*
                        index_coefficient(total_wavefunction,creat_i_ospin(i,total_wavefunction[h][0][k],spin)[0],coefficients)*
                        index_coefficient(total_wavefunction,annih_i_ospin(i,total_wavefunction[h][1][l],spin)[0],coefficients))
    return x 
#print(s1i_creation_s2i_annihilation(1,w,c,'a'))

#defining hopping term
def hopping(total_wavefunction,coefficients):
    t=0
    for i in range(4):
        B=[3512,9679,4653,8472,6562]
        t+=B[i-1]*s1i_creation_s2i_annihilation(1,total_wavefunction,coefficients,'a')
        t+=B[i-1]*s1i_creation_s2i_annihilation(1,total_wavefunction,coefficients,'b')
    return t
#print(hopping(total_wavefunction,coefficients))
hopping_energy=3579.5466666666716

In [34]:
#total energy with new algorithm
E=0
for i in range(4):
    J=[2656,2743,2151, 1756,395]
    B=[3512,9679,4653,8472,6562]
    delta=[0,1536,4433,6167,6167]
    E+=(J[i+1]*(s1iz_s2iz(i,w,c)+0.5*
            (s1i_raising_s2i_lowering(i,w,c)+
             s1i_lowering_s2i_raising(i,w,c)))+
             delta[i+1]*ligand_field_splitting_i(i,w,c)+
             B[i-1]*(s1i_creation_s2i_annihilation(1,w,c,'a')+
                    s1i_creation_s2i_annihilation(1,w,c,'b')))
print(E)
Energy1=44402.22376303869 #erroneous due to incorrect sign of CG coefficient
Energy_True=40049.5706170311

40049.5706170311
